# Boceto E13


## Comentarios a discutir:
1. Como interpretan la pergunta 'Analyze the writing patterns of each student'?, seria algo tipo hacer un top de palabras por estudiante?
2. Para la tokenización "entrené" y llevé a matrices cada columna de preguntas por aparte, suponiendo que las pabaras que fueron importantes en el contexto de una pregunta pueden no serlo en el contexto de otra.
3. Hay que predecir sector laboral, pero tiene bastantes valores unicos, a fuerza definí 3 grupos, podemos conversarlos.
4. De cara a los modelos y al 'CountVectorizer' tenemos bastantes opciones donde calibrar hiperparametros, dado el tema de fechas de entrega y otros pendientes que tanto queremos profundizar?


# Exercise 13

The result will be evaluated from a report in Jupyter, which must be found in a public GitHub repository. The project must be carried out in the groups assigned in class. Use clear and rigorous procedures. Due date: July 29, 2021, 11:59 am (NOON), through Bloque Neón + (Upload repository link).

## Analyze class homeworks


In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import cross_val_score 
from sklearn.tree import DecisionTreeClassifier     
from sklearn.ensemble import RandomForestClassifier

from nltk.stem import WordNetLemmatizer

import nltk
nltk.corpus.stopwords.words('spanish')
nltk.download('wordnet') 

from sklearn.naive_bayes import MultinomialNB


from nltk.corpus import stopwords
#nltk.download('stopwords')


%matplotlib inline
import matplotlib.pyplot as plt

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sebtc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_excel('Datasets/Consolidado_respuestas_escribir_v2.xlsx')
data.columns = ['ID', 'Genero', 'Pregrado', 'Sector_trabajo','Edad','E1','E4','E6','E8','E10','E12']

In [3]:
E1_text = data['E1'].fillna("Sin texto")
E4_text = data['E4'].fillna("Sin texto")
E6_text = data['E6'].fillna("Sin texto")
E8_text = data['E8'].fillna("Sin texto")
E10_text = data['E10'].fillna("Sin texto")
E12_text = data['E12'].fillna("Sin texto")

In [4]:
print(data['Genero'].value_counts())
y_genero = data['Genero']

M    28
F    10
Name: Genero, dtype: int64


In [5]:
data['Sector_trabajo']=data['Sector_trabajo'].str.lower()
data['Sector_trabajo']=np.where(data['Sector_trabajo'].str.contains('fin|banc'),'Financiero',data['Sector_trabajo'])
data['Sector_trabajo']=np.where(data['Sector_trabajo'].str.contains('eléc|comu|estructu|ingen|oper'),'Ingenieria',data['Sector_trabajo'])
data['Sector_trabajo']=np.where(data['Sector_trabajo'].str.contains('Fin|Ingen'),data['Sector_trabajo'],'Otros')
print(data['Sector_trabajo'].value_counts())
y_sector = data['Sector_trabajo']

Financiero    24
Otros          9
Ingenieria     5
Name: Sector_trabajo, dtype: int64


In [6]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word,pos='v') for word in words]
spanish_stopwords = stopwords.words('spanish')
vect = CountVectorizer(ngram_range=(1,2),lowercase=True,stop_words=spanish_stopwords,max_features=100,min_df=2)#,analyzer=split_into_lemmas,)
#vect = TfidfVectorizer(analyzer=split_into_lemmas,ngram_range=(1,5),lowercase=True,stop_words=spanish_stopwords),max_features=100,min_df=2)

In [7]:
vect.fit(E1_text)
E1_text_dtm = vect.transform(E1_text)
vect.fit(E4_text)
E4_text_dtm = vect.transform(E4_text)
vect.fit(E6_text)
E6_text_dtm = vect.transform(E6_text)
vect.fit(E8_text)
E8_text_dtm = vect.transform(E8_text)
vect.fit(E10_text)
E10_text_dtm = vect.transform(E10_text)
vect.fit(E12_text)
E12_text_dtm = vect.transform(E12_text)
#variables predictoras
E1_text_dtm.shape,E4_text_dtm.shape,E6_text_dtm.shape,E8_text_dtm.shape,E10_text_dtm.shape,E12_text_dtm.shape

((38, 100), (38, 100), (38, 100), (38, 100), (38, 100), (38, 100))

In [8]:
tokens_pred=np.c_[E1_text_dtm.todense(),E4_text_dtm.todense(),E6_text_dtm.todense(),E8_text_dtm.todense(),E10_text_dtm.todense(),E12_text_dtm.todense()]
tokens_pred.shape

(38, 600)

# Exercise 13.1

Analyze the writing patterns of each student

# Exercise 13.2

Create a classifier to predict the gender of each student

In [9]:
rfCLF = RandomForestClassifier(random_state=1)
pd.Series(cross_val_score(rfCLF, tokens_pred, y_genero, cv=10)).describe()

count    10.000000
mean      0.733333
std       0.035136
min       0.666667
25%       0.750000
50%       0.750000
75%       0.750000
max       0.750000
dtype: float64

# Exercise 13.3

Create a classifier to predict the industry in which each student works

In [10]:
nb = MultinomialNB()
pd.Series(cross_val_score(nb, tokens_pred, y_sector, cv=5)).describe()

count    5.000000
mean     0.528571
std      0.108327
min      0.428571
25%      0.500000
50%      0.500000
75%      0.500000
max      0.714286
dtype: float64